In [152]:
# Import packages.
import pandas as pd
from datetime import timedelta

In [153]:
# Read in the data.
crypto = pd.read_pickle("../data/crypto_3_9.pkl")
tweets = pd.read_pickle("../data/tweets_3_9.pkl")

In [154]:
# Define a function.
def get_max(min_stamp, price):
    max_stamp = min_stamp + timedelta(hours=3)
    mask = (crypto["stamp"] > min_stamp) & (crypto["stamp"] < max_stamp)
    sub_df = crypto.loc[mask]
    max_price = sub_df.price.max()

    # Create the difference.
    difference = (max_price - price) / price
    
    # Return. THIS IS IMPORTANT.
    if difference > .0025:
        return 1
    else:
        return 0

# Apply the function.
crypto["increase"] = crypto.apply(lambda x: get_max(x[0], x[2]), axis=1)
crypto.head(3)

,stamp,crypto,price,increase
0,2022-02-16 09:13:00,BTC/USD,43677.5,0
1,2022-02-16 09:14:00,BTC/USD,43677.5,0
2,2022-02-16 09:15:00,BTC/USD,43677.5,0


In [155]:
# Join the data.
training_data = tweets.merge(crypto, left_on="time", right_on="stamp")
training_data = training_data.set_index("tweet_id")
training_data = training_data.loc[:, ["time", "tweet", "increase"]]
training_data.tail()

,time,tweet,increase
tweet_id,,,
348440,2022-02-17 19:26:00,just in - fbi to form national cryptocurrency ...,0
348441,2022-02-17 19:26:00,bitcoin is a bank that will never lock you out...,0
348442,2022-02-17 19:26:00,$btc double top validates at 41.5k. lots of be...,0
348443,2022-02-17 19:26:00,signal spreading - opt out with bitcoin,0
348444,2022-02-17 19:26:00,bitcoin doesnt solve this,0


In [156]:
# BERT, in chunks.

(338924, 3)